<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Разработка-и-обучение-нескольких-моделей-для-прогноза-прочности-при-растяжении" data-toc-modified-id="Разработка-и-обучение-нескольких-моделей-для-прогноза-прочности-при-растяжении-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Разработка и обучение нескольких моделей для прогноза прочности при растяжении</a></span><ul class="toc-item"><li><span><a href="#Прогнозируем-прочность-при-растяжении" data-toc-modified-id="Прогнозируем-прочность-при-растяжении-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Прогнозируем прочность при растяжении</a></span></li></ul></li></ul></div>

# Продолжение кода по композитным материалам

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import plotly.express as px
import tensorflow as tf
import sklearn

from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error, mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn import preprocessing
from sklearn.preprocessing import Normalizer, LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
import pickle

from tensorflow import keras as keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, Activation
from pandas import read_excel, DataFrame, Series
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from tensorflow.keras.models import Sequential
from numpy.random import seed
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_excel('df_clean.xlsx')
df.head()

,Unnamed: 0,Соотношение матрица-наполнитель,"Плотность, кг/м3","модуль упругости, ГПа","Количество отвердителя, м.%","Содержание эпоксидных групп,%_2","Температура вспышки, С_2","Поверхностная плотность, г/м2","Модуль упругости при растяжении, ГПа","Прочность при растяжении, МПа","Потребление смолы, г/м2",Угол нашивки,Шаг нашивки,Плотность нашивки
0,1,1.857143,2030.0,738.736842,50.00,23.750000,284.615385,210.0,70.0,3000.0,220.0,0,4.0,60.0
1,3,1.857143,2030.0,738.736842,129.00,21.250000,300.000000,210.0,70.0,3000.0,220.0,0,5.0,47.0
2,4,2.771331,2030.0,753.000000,111.86,22.267857,284.615385,210.0,70.0,3000.0,220.0,0,5.0,57.0
3,5,2.767918,2000.0,748.000000,111.86,22.267857,284.615385,210.0,70.0,3000.0,220.0,0,5.0,60.0
4,6,2.569620,1910.0,807.000000,111.86,22.267857,284.615385,210.0,70.0,3000.0,220.0,0,5.0,70.0


In [ ]:
#Удаляем первый столбец, в котором только нумерация
df.drop(df.columns[df.columns.str.contains('unnamed', case=False)],
           axis=1,
           inplace=True)



In [ ]:
df.head()

,Соотношение матрица-наполнитель,"Плотность, кг/м3","модуль упругости, ГПа","Количество отвердителя, м.%","Содержание эпоксидных групп,%_2","Температура вспышки, С_2","Поверхностная плотность, г/м2","Модуль упругости при растяжении, ГПа","Прочность при растяжении, МПа","Потребление смолы, г/м2",Угол нашивки,Шаг нашивки,Плотность нашивки
0,1.857143,2030.0,738.736842,50.00,23.750000,284.615385,210.0,70.0,3000.0,220.0,0,4.0,60.0
1,1.857143,2030.0,738.736842,129.00,21.250000,300.000000,210.0,70.0,3000.0,220.0,0,5.0,47.0
2,2.771331,2030.0,753.000000,111.86,22.267857,284.615385,210.0,70.0,3000.0,220.0,0,5.0,57.0
3,2.767918,2000.0,748.000000,111.86,22.267857,284.615385,210.0,70.0,3000.0,220.0,0,5.0,60.0
4,2.569620,1910.0,807.000000,111.86,22.267857,284.615385,210.0,70.0,3000.0,220.0,0,5.0,70.0


### Разработка и обучение нескольких моделей для прогноза прочности при растяжении

На размеченных данных применеяются задачи регрессии и классификации, и так как нам нужно предсказать непрерывную переменную, то мы будем разрабатывать модели регрессии.
Будем учитывать полученное задание, что при построении моделей нужно провести поиск гиперпараметров модели с помощью поиска по сетке с перекрестной проверкой, количество блоков равно 10.

Сформируем выборки и посмотрим, что получилось

#### Прогнозируем прочность при растяжении

После всех подготовительных работ переходим к процессу создания, обучения моделей. Мы будем использовать в Python библиотеку Scikit-Learn.
В качестве базового уровня предскажем медианное значение цели на обучающем наборе для всех примеров в тестовом наборе. В качестве метрики возьмём среднюю абсолютную ошибку (mae) в прогнозах. Для обучения используем 70 % данных, а для тестирования — 30 %.

In [ ]:
X = df.drop(columns=['Прочность при растяжении, МПа'],
            axis=1)

In [ ]:
X

,Соотношение матрица-наполнитель,"Плотность, кг/м3","модуль упругости, ГПа","Количество отвердителя, м.%","Содержание эпоксидных групп,%_2","Температура вспышки, С_2","Поверхностная плотность, г/м2","Модуль упругости при растяжении, ГПа","Потребление смолы, г/м2",Угол нашивки,Шаг нашивки,Плотность нашивки
0,1.857143,2030.000000,738.736842,50.000000,23.750000,284.615385,210.000000,70.000000,220.000000,0,4.000000,60.000000
1,1.857143,2030.000000,738.736842,129.000000,21.250000,300.000000,210.000000,70.000000,220.000000,0,5.000000,47.000000
2,2.771331,2030.000000,753.000000,111.860000,22.267857,284.615385,210.000000,70.000000,220.000000,0,5.000000,57.000000
3,2.767918,2000.000000,748.000000,111.860000,22.267857,284.615385,210.000000,70.000000,220.000000,0,5.000000,60.000000
4,2.569620,1910.000000,807.000000,111.860000,22.267857,284.615385,210.000000,70.000000,220.000000,0,5.000000,70.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
931,2.271346,1952.087902,912.855545,86.992183,20.123249,324.774576,209.198700,73.090961,125.007669,1,9.076380,47.019770
932,3.444022,2050.089171,444.732634,145.981978,19.599769,254.215401,350.660830,72.920827,117.730099,1,10.565614,53.750790
933,3.280604,1972.372865,416.836524,110.533477,23.957502,248.423047,740.142791,74.734344,236.606764,1,4.161154,67.629684
934,3.705351,2066.799773,741.475517,141.397963,19.246945,275.779840,641.468152,74.042708,197.126067,1,6.313201,58.261074


In [ ]:
y = df['Прочность при растяжении, МПа']

In [ ]:
y

1       3000.000000
3       3000.000000
4       3000.000000
5       3000.000000
6       3000.000000
           ...     
1018    2387.292495
1019    2360.392784
1020    2662.906040
1021    2071.715856
1022    2856.328932
Name: Прочность при растяжении, МПа, Length: 936, dtype: float64

In [ ]:
# разбиваем на тестовую и тренировочную выборки, выделяя признаки и целевую переменную

X_train, X_test, y_train, y_test = train_test_split(X,
                                                            y,
                                                            test_size=0.3,
                                                            random_state=42)

In [ ]:
y_train.shape

(655,)

In [ ]:
#Функция для сравнения результатов предсказаний с моделью, выдающей среднее значение по тестовой выборке
def mean_model(y_test):
    return [np.mean(y_test) for _ in range(len(y_test))]
y_pred_mean = mean_model(y_test)

Линейная регрессия

In [ ]:
#Создадим переменную для названия всех столбцов. Это нам пригодится при построении моделей. И перейдем к визуализации данных
df.columns
#column_names = ["Соотношение матрица-наполнитель","Плотность, кг/м3","модуль упругости, ГПа","Количество отвердителя, м.%",
#         "Содержание эпоксидных групп,%_2","Температура вспышки, С_2","Поверхностная плотность, г/м2",
#         "Модуль упругости при растяжении, ГПа","Прочность при растяжении, МПа","Потребление смолы, г/м2",
#        "Угол нашивки, град","Шаг нашивки","Плотность нашивки"]
column_names = df.columns

Линейная регрессия

In [ ]:
lr = make_pipeline(StandardScaler(),LinearRegression()) 
lr.fit(X_train, y_train) 
y_pred_lr = lr.predict(X_test) 
mae_lr = mean_absolute_error(y_pred_lr, y_test) 

print('Linear Regression Results Train:') # Оценка для тренировочной выборки 
print("Train score: {:.2f}".format(lr.score(X_train, y_train))) 
print('Linear Regression Results:')
print("Test score: {:.2f}".format(lr.score(X_test, y_test))) # Оценка для тестовой выборки

Linear Regression Results Train:
Train score: 0.03
Linear Regression Results:
Test score: -0.05


In [ ]:
pickle.dump(lr, open('model.pkl','wb'))

Decision Tree

In [ ]:
#Деревья решений - Decision Tree Regressor 

dtr = make_pipeline(StandardScaler(),DecisionTreeRegressor())
dtr.fit(X_train, y_train)
y_pred_dtr = dtr.predict(X_test)
mae_dtr = mean_absolute_error(y_pred_dtr, y_test)


print('Decision Tree Regressor Results Train:')
print("Test score: {:.2f}".format(dtr.score(X_train, y_train)))# Оценка для тренировочной выборки
print('Decision Tree Regressor Results:')
print('DTR_MAE: ', round(mean_absolute_error(y_test, y_pred_dtr)))


print("Test score: {:.2f}".format(dtr.score(X_test, y_test)))# Оценка для тестовой выборки

Decision Tree Regressor Results Train:
Test score: 1.00
Decision Tree Regressor Results:
DTR_MAE:  556
Test score: -1.40


Метод случайного леса - Random Forest Regressor

In [ ]:
#Метод случайного леса
rfr = make_pipeline(StandardScaler(),RandomForestRegressor(n_estimators=15,max_depth=9, random_state=45))
rfr.fit(X_train, y_train.values)
y_pred_forest = rfr.predict(X_test)
mae_rfr = mean_absolute_error(y_pred_forest, y_test)

print('Random Forest Regressor Results Train:')
print("Test score: {:.2f}".format(rfr.score(X_train, y_train))) # Оценка для тренировочной выборки
print('Random Forest Regressor Results:')
print('RF_MAE: ', round(mean_absolute_error(y_test, y_pred_forest)))
print("Test score: {:.2f}".format(rfr.score(X_test, y_test))) # Оценка для тестовой выборки

Random Forest Regressor Results Train:
Test score: 0.61
Random Forest Regressor Results:
RF_MAE:  374
Test score: -0.10


Метод К ближайших соседей

In [ ]:
# Метод К ближайших соседей - K Neighbors Regressor - 5
knn = make_pipeline(StandardScaler(),KNeighborsRegressor(n_neighbors=5))
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
mae_knr = mean_absolute_error(y_pred_knn, y_test)

print('K Neighbors Regressor  Results Train:')
print("Test score: {:.2f}".format(knn.score(X_train, y_train)))# Оценка для тренировочной выборки
print('K Neighbors Regressor  Results:')
print('KNN_MAE: ', round(mean_absolute_error(y_test, y_pred_knn)))
print("Test score: {:.2f}".format(knn.score(X_test, y_test)))# Оценка для тестовой выборки

K Neighbors Regressor  Results Train:
Test score: 0.20
K Neighbors Regressor  Results:
KNN_MAE:  392
Test score: -0.26


Метода градиентного бустинга - Gradient Boosting Regressor

In [ ]:
gbr = make_pipeline(StandardScaler(), GradientBoostingRegressor())
gbr.fit(X_train, y_train)
y_pred_gbr = gbr.predict(X_test)
mae_gbr = mean_absolute_error(y_pred_gbr, y_test)
mse_gbr_elast = mean_squared_error(y_test,y_pred_gbr)

print('Gradient Boosting Regressor Results Train:')
print("Test score: {:.2f}".format(gbr.score(X_train, y_train))) # Оценка тренировочной выборки
print('Gradient Boosting Regressor Results:')
print('GBR_MAE: ', round(mean_absolute_error(y_test, y_pred_gbr)))
print("Test score: {:.2f}".format(gbr.score(X_test, y_test)))# Оценка для тестовой выборки

Gradient Boosting Regressor Results Train:
Test score: 0.54
Gradient Boosting Regressor Results:
GBR_MAE:  381
Test score: -0.12


сравним наши модели по метрике МАЕ

In [ ]:
#сравним наши модели по метрике МАЕ
mae_df = {'Регрессор': ['RandomForest', 'Linear Regression', 'KNeighbors', 'DecisionTree', 'Gradient Boosting Regressor'], 'MAE': [mae_rfr, mae_lr, mae_knr, mae_dtr, mae_gbr]} 

mae_df = pd.DataFrame(mae_df)

In [ ]:
mae_df

,Регрессор,MAE
0,RandomForest,371.862402
1,Linear Regression,363.144068
2,KNeighbors,392.394492
3,DecisionTree,541.022823
4,Gradient Boosting Regressor,382.081859


Модель линейная регрессия показала наилучшие показатели mae. Проведем поиск  по сетке гиперпараметров с перекрестной проверкой, количество блоков равно 10 (cv = 10), для 
модели линейной регрессии 

In [ ]:
ridge=Ridge()
parameters={'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,35,40,45,50,55,100]}
ridge_regressor=GridSearchCV(ridge,parameters,scoring='neg_mean_squared_error',cv=10)
ridge_regressor.fit(X_train,y_train)

GridSearchCV(cv=10, estimator=Ridge(),
             param_grid={'alpha': [1e-15, 1e-10, 1e-08, 0.001, 0.01, 1, 5, 10,
                                   20, 30, 35, 40, 45, 50, 55, 100]},
             scoring='neg_mean_squared_error')

In [ ]:
print(ridge_regressor.best_params_)
print(ridge_regressor.best_score_)

{'alpha': 100}
-223785.8626644357


In [ ]:
prediction_ridge=ridge_regressor.predict(X_test)

In [ ]:
prediction_ridge

In [ ]:
print('lr_MAE_ridge: ', round(mean_absolute_error(y_test, prediction_ridge),4))

lr_MAE_ridge:  363.0528


Оптимизация с помощью GridSearch не улучшила показателей модели.